In [6]:
import requests
import os

In [68]:
import wget
import calendar

#### Download Day's Worth of Data

In [45]:
download_urls = []
BASE = 'http://data.githubarchive.org/'
year = 2016
month = '02'
day_str = '03' # set to current day

In [16]:
for hour in range(0, 24):
    dl_url = str(BASE) + str(year) + "-" + str(month) + "-" + day_str + "-" + str(hour) + ".json.gz"
    download_urls.append(dl_url) 

In [18]:
def wget_github(url):
    try:
        filename = wget.download(url)
    except Exception as ex:
        template = "An exception of type {0} occured. Arguments:\n{1!r}"
        message = template.format(type(ex).__name__, ex.args)
        print(message)
        with open("wget_log.txt", "a") as log:
            log.write("ERROR IN: " + url + " with message: \n")
            log.write(message)
            log.write("\n")
            log.close()

In [ ]:
print('beginning dl')
for i, url in enumerate(download_urls):
    print('starting:' + url)
    wget_github(url)
print('finished')

In [25]:
download_urls[0].split('/')[3]

'2016-02-01-1.json.gz'

#### Parse Data:
load gzjson files into pandas dataframe and 
process commit events and load language/repo info
process watch and fork events 

In [44]:
import ujson
import gzip
import matplotlib
import pandas
import os
import re
import time
import pprint

In [43]:
download_urls 

['http://data.githubarchive.org/2016-02-01-0.json.gz',
 'http://data.githubarchive.org/2016-02-01-1.json.gz',
 'http://data.githubarchive.org/2016-02-01-2.json.gz',
 'http://data.githubarchive.org/2016-02-01-3.json.gz',
 'http://data.githubarchive.org/2016-02-01-4.json.gz',
 'http://data.githubarchive.org/2016-02-01-5.json.gz',
 'http://data.githubarchive.org/2016-02-01-6.json.gz',
 'http://data.githubarchive.org/2016-02-01-7.json.gz',
 'http://data.githubarchive.org/2016-02-01-8.json.gz',
 'http://data.githubarchive.org/2016-02-01-9.json.gz',
 'http://data.githubarchive.org/2016-02-01-10.json.gz',
 'http://data.githubarchive.org/2016-02-01-11.json.gz',
 'http://data.githubarchive.org/2016-02-01-12.json.gz',
 'http://data.githubarchive.org/2016-02-01-13.json.gz',
 'http://data.githubarchive.org/2016-02-01-14.json.gz',
 'http://data.githubarchive.org/2016-02-01-15.json.gz',
 'http://data.githubarchive.org/2016-02-01-16.json.gz',
 'http://data.githubarchive.org/2016-02-01-17.json.gz',
 '

In [ ]:
def parse_push_events(file):
    repo_id = []
    repo_name = []
    created_at = []
    actor_id = []
    actor_login = []
    with gzip.open(file) as f:
        for i, line in enumerate(f):
            json_data = ujson.loads(line)
            if (json_data['type'] == 'PushEvent'):
                repo_id = json_data['repo']['id']
                repo_name = json_data['repo']['name']
                created_at = json_data['created_at']
                actor_id = json_data['actor']['id']
                actor_login = json_data['actor']['login']
        f.close()
    
    

In [70]:
push_events = []
def parse_push_events(json_data):
    result = {'event': 'PushEvent'}
    result['repo_id'] = json_data['repo']['id']
    result['repo_name'] = json_data['repo']['name']
    result['created_at'] = json_data['created_at']
    result['actor_id'] = json_data['actor']['id']
    result['actor_login'] = json_data['actor']['login']
    result['org'] = False
    result['org_id'] = None
    if 'org' in json_data:
        result['org'] = True
        result['org_id'] = json_data['org']['id']
    return result

In [71]:
with gzip.open('2016-02-01-23.json.gz') as f:
    for i, line in enumerate(f):
        json_data = ujson.loads(line)
        
        if (json_data['type'] == 'PushEvent'):
            push_events.append( parse_push_events(json_data) )
            
    
    f.close()

In [83]:
pull_request_review_comment_events = []
def parse_pull_request_review_comment_events(json_data):
    result = {'event': 'PullRequestReviewCommentEvent'}
    result['language'] = json_data['payload']['pull_request']['base']['repo']['language'] 
    result["stargazers_count"] = json_data['payload']['pull_request']['base']['repo']["stargazers_count"] 
    result['repo_id'] = json_data['payload']['pull_request']['base']['repo']['id'] 
    result['repo_name'] = json_data['payload']['pull_request']['base']['repo']['full_name'] 
    result['created_at'] = json_data['created_at']
    result['actor_id'] = json_data['actor']['id']
    result['actor_login'] = json_data['actor']['login']
    result['org'] = False
    result['org_id'] = None
    if 'org' in json_data:
        result['org'] = True
        result['org_id'] = json_data['org']['id']
    return result

In [84]:
with gzip.open('2016-02-01-23.json.gz') as f:
    for i, line in enumerate(f):
        json_data = ujson.loads(line)
        
        if (json_data['type'] == 'PullRequestReviewCommentEvent'):
            pull_request_review_comment_events.append( parse_pull_request_review_comment_events(json_data) )
            
    
    f.close()

In [85]:
pull_request_review_comment_events

[{'actor_id': 685609,
  'actor_login': 'NachoSoto',
  'created_at': '2016-02-01T23:00:00Z',
  'event': 'PullRequestReviewCommentEvent',
  'language': None,
  'org': True,
  'org_id': 10639145,
  'repo_id': 45497910,
  'repo_name': 'apple/swift-evolution',
  'stargazers_count': 2625},
 {'actor_id': 8660335,
  'actor_login': 'brendan-hofmann',
  'created_at': '2016-02-01T23:00:07Z',
  'event': 'PullRequestReviewCommentEvent',
  'language': 'Java',
  'org': True,
  'org_id': 2386734,
  'repo_id': 15173563,
  'repo_name': 'codice/ddf',
  'stargazers_count': 32},
 {'actor_id': 8763518,
  'actor_login': 'cdart2',
  'created_at': '2016-02-01T23:00:12Z',
  'event': 'PullRequestReviewCommentEvent',
  'language': 'JavaScript',
  'org': True,
  'org_id': 14594176,
  'repo_id': 42892349,
  'repo_name': 'project415x/project415x.github.io',
  'stargazers_count': 0},
 {'actor_id': 748971,
  'actor_login': 'enricostano',
  'created_at': '2016-02-01T23:00:13Z',
  'event': 'PullRequestReviewCommentEvent

In [86]:
pull_request_events = []
def parse_pull_request_events(json_data):
    result = {'event': 'PullRequestEvent'}
    result['repo_id'] = json_data['payload']['pull_request']['base']['repo']['id'] 
    result['repo_name'] = json_data['payload']['pull_request']['base']['repo']['name']
    result['description'] = json_data['payload']['pull_request']['base']['repo']['description']
    result['language'] = json_data['payload']['pull_request']['base']['repo']['language']
    result["stargazers_count"] = json_data['payload']['pull_request']['base']['repo']["stargazers_count"] 
    result['created_at'] = json_data['created_at']
    result['actor_id'] = json_data['actor']['id']
    result['actor_login'] = json_data['actor']['login']
    result['org'] = False
    result['org_id'] = None
    if 'org' in json_data:
        result['org'] = True
        result['org_id'] = json_data['org']['id']
    return result

In [87]:
with gzip.open('2016-02-01-23.json.gz') as f:
    for i, line in enumerate(f):
        json_data = ujson.loads(line)
        
        if (json_data['type'] == 'PullRequestEvent'):
            pull_request_events.append( parse_pull_request_events(json_data) )
            
    
    f.close()

In [88]:
pull_request_events

[{'actor_id': 10633253,
  'actor_login': 'ztiffany',
  'created_at': '2016-02-01T23:00:05Z',
  'description': 'Open Fabric Interfaces',
  'event': 'PullRequestEvent',
  'language': 'C',
  'org': True,
  'org_id': 10637979,
  'repo_id': 29607348,
  'repo_name': 'libfabric-cray',
  'stargazers_count': 5},
 {'actor_id': 1793177,
  'actor_login': '2ndkauboy',
  'created_at': '2016-02-01T23:00:07Z',
  'description': 'Alle deutschsprachigen WP Meetups - in einem Widget.',
  'event': 'PullRequestEvent',
  'language': 'PHP',
  'org': True,
  'org_id': 6129203,
  'repo_id': 30495341,
  'repo_name': 'wpmg-widget',
  'stargazers_count': 2},
 {'actor_id': 154673,
  'actor_login': 'johann8384',
  'created_at': '2016-02-01T23:00:16Z',
  'description': 'Data collection framework for OpenTSDB',
  'event': 'PullRequestEvent',
  'language': 'Python',
  'org': True,
  'org_id': 2086220,
  'repo_id': 1189586,
  'repo_name': 'tcollector',
  'stargazers_count': 318},
 {'actor_id': 2996465,
  'actor_login': 

In [ ]:
def parse_watch_events(file):
    repository = []
    repo_owner = []
    starer = []
    time = []
    with gzip.open(file) as f:
        for i, line in enumerate(f):
            json_data = ujson.loads(line)
            if (json_data['type'] == 'WatchEvent'):
                repo_name = json_data['repo']['name']
                repository.append(repo_name)
                repo_owner.append(repo_name.split('/')[0])
                starer.append(json_data['actor']['login'])
                time.append(json_data['created_at'])
        f.close()
    return({"repository": repository, "repo_owner": repo_owner, "starer": starer, "time": time})
        

In [ ]:
# merge a days worth
for i, url in enumerate(download_urls[0]):
    file_name = url.split('/')[-1]
    with gzip.open(file_name) as f:
        for i, line in enumerate(f):
            json_data = ujson.loads(line)
            # logic for splitting up into requisite tables
            types.append(json_data['type'])
        f.close()

In [ ]:
# commit events